In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


: 

## Let's load the Boston House Pricing Dataset

In [ ]:
df = pd.read_csv('./HousingData.csv')
df.head()

CRIM - per capita crime rate by town

ZN - proportion of residential land zoned for lots over 25,000 sq.ft.

INDUS - proportion of non-retail business acres per town.

CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)

NOX - nitric oxides concentration (parts per 10 million)

RM - average number of rooms per dwelling

AGE - proportion of owner-occupied units built prior to 1940

DIS - weighted distances to five Boston employment centres

RAD - index of accessibility to radial highways

TAX - full-value property-tax rate per $10,000

PTRATIO - pupil-teacher ratio by town

B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

LSTAT - % lower status of the population

MEDV - Median value of owner-occupied homes in $1000's

In [ ]:
df.info()

In [ ]:
## Let's check the description of the dataset
df.describe()


In [ ]:
print(df.columns)

In [ ]:
df=df[["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]]
df

In [ ]:
df['MEDV']

## Preparing the Dataset

In [ ]:
# Separate X (features) and y (target)
X = df.drop(columns='MEDV')  
y = df['MEDV']               

# Get feature names
feature_names = X.columns.tolist()


dataset = pd.DataFrame(X.values, columns=feature_names)

print(dataset.head())


In [ ]:
dataset['Price']=y.values
dataset.head()

In [ ]:
dataset.info()

In [ ]:
## Summarizing the dataset
dataset.describe()

In [ ]:
## Check the missing values
dataset.isnull().sum()

In [ ]:
# Fill numerical columns with mean
for col in ['CRIM', 'ZN','INDUS', 'AGE', 'LSTAT']:
    dataset[col] = dataset[col].fillna(dataset[col].mean())

# Fill categorical column with mode
dataset['CHAS'] = dataset['CHAS'].fillna(dataset['CHAS'].mode()[0])

# Now check again:
print(dataset.isnull().sum())


In [ ]:
## Exploratory Data Analysis (EDA)
## Correlation
dataset.corr()

In [ ]:
plt.scatter(dataset['CRIM'], dataset['Price'])
plt.xlabel('Crime Rate')
plt.ylabel('Price')
plt.title('Crime Rate vs Price')
plt.show()

In [ ]:
plt.scatter(dataset['RM'], dataset['Price'])
plt.xlabel('Average Number of Rooms')
plt.ylabel('Price') 
plt.title('Average Number of Rooms vs Price')
plt.show()

In [ ]:
import seaborn as sns
sns.regplot(x='RM', y='Price', data=dataset, lowess=True, line_kws={'color': 'red', 'lw': 1})
# we can see that the average number of rooms has a positive correlation with the price of the house.
# The more rooms, the higher the price.
# The regression line is a good fit for the data points.
# The data points are not perfectly linear, but the trend is clear.

In [ ]:
sns.regplot(x='LSTAT', y='Price', data=dataset, lowess=True, line_kws={'color': 'red', 'lw': 1})

In [ ]:
sns.regplot(x="CHAS", y="Price", data=dataset, lowess=True, line_kws={'color': 'red', 'lw': 1})

In [ ]:
sns.regplot(x="PTRATIO", y="Price", data=dataset, lowess=True, line_kws={'color': 'red', 'lw': 1})

In [ ]:
## Independant and Dependent features

dataset
# price is the dependent feature and the rest are independent features.

In [ ]:
# We can also check the correlation between the features and the target variable (Price) using a heatmap.
# The heatmap will show the correlation between all the features and the target variable.

plt.figure(figsize=(12, 8))
sns.heatmap(dataset.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
X=dataset.iloc[:, :-1]
y=dataset.iloc[:, -1]
X.head()

In [ ]:
y

In [ ]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
X_test
# We're not going to use the test set for training
# We will use it to evaluate the model performance after training.
# We will use the training set to train the model and the test set to evaluate the model performance.

In [ ]:
print("Skewness: ", dataset.skew())
print("Kurtosis: ", dataset.kurtosis())



In [ ]:
for col in dataset.columns:
    if col != 'Price':  # don't plot target
        plt.figure(figsize=(8, 5))
        sns.histplot(dataset[col], kde=True, bins=30)
        plt.title(f"Distribution of {col}")
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()


In [ ]:
## Standaridze the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# in X_test_scaled we use transform instead of fit_transform because we don't want to fit the scaler again on the test set.
# This is done to avoid data leakage from the test set into the training set.

In [ ]:
X_train

In [ ]:
X_test

## Model Training

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
regression = LinearRegression()

In [ ]:
regression.fit(X_train_scaled, y_train)
# The model is trained now.
# We can check the coefficients of the model to see the importance of each feature.
print("Coefficients: ", regression.coef_)
# The coefficients represent the change in the target variable (Price) for a one unit change in the feature.
# A positive coefficient means that the feature has a positive impact on the target variable (Price).
# A negative coefficient means that the feature has a negative impact on the target variable (Price).
# The larger the absolute value of the coefficient, the more important the feature is in the model.

In [ ]:
print("Intercept: ", regression.intercept_)
# The intercept is the value of the target variable (Price) when all the features are 0.
# In this case, it doesn't have much meaning because the features are standardized.

In [ ]:
## on which parameter the model is trained on
print("Parameters: ", regression.get_params())

In [ ]:
## Prediction with Test Data
reg_pred=regression.predict(X_test_scaled)

In [ ]:
reg_pred

## Assumptions

In [ ]:
## plot a scatter plot of the predicted values vs the actual values
plt.scatter(y_test, reg_pred)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Actual Price vs Predicted Price')
plt.show()
# The plot is linear, which means that the model is a good fit for the data.
# The points are close to the line, which means that the model is a good fit for the data.

In [ ]:
## Residuals (Errors)
residuals = y_test - reg_pred
residuals

In [ ]:
## Plot the residuals
sns.displot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()
# There are outliers in the residuals
# We stil have a normal distribution of the residuals.
# The residuals are normally distributed, which means that the model is a good fit for the data.

In [ ]:
## Scatter plot with respect to prediction and residuals
plt.scatter(reg_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Price')
plt.ylabel('Residuals')
# They're scattered uniformly around 0, which means that the model is a good fit for the data.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print("Mean Absolute Error: ", mean_absolute_error(y_test, reg_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, reg_pred))
print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(y_test, reg_pred)))


In [ ]:
from sklearn.metrics import r2_score
print("R2 Score: ", r2_score(y_test, reg_pred))
# The R2 score is 0.65, which means that the model explains 65% of the variance in the target variable (Price).

## R square and adjusted R square

$$
R^2 = 1 - \frac{SSR}{SST}
$$

Where:
- \( R^2 \) = Coefficient of determination
- \( SSR \) = Sum of Squares of Residuals
- \( SST \) = Total Sum of Squares


In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y_test, reg_pred)
print("R2 Score: ", score)
# The R2 score is 0.65, which means that the model explains 65% of the variance in the target variable (Price).

##### Adjusted R2 = 1 - [(1-R2)*(n-1)/(n-k-1)]
Where: 
- R2: The R2 of the model
- n: The number of observations
- k: The number of predictors

In [ ]:
#display Adjusted R-squared 
1- (1-score)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# log-transform skewed features
skewed_feats = ['CRIM','ZN','CHAS','B']
log_tf = FunctionTransformer(np.log1p, validate=False)

# column transformer
preproc = ColumnTransformer([
    ('log',     log_tf,        skewed_feats),
    ('passthru','passthrough', [c for c in X_train.columns if c not in skewed_feats])
])

# full pipeline
from tempfile import mkdtemp
cachedir = mkdtemp()

pipeline = Pipeline([
    ('pre',  preproc),
    #('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),
    ('scale',StandardScaler()),
    ('reg',  RandomForestRegressor(
                  n_estimators=200,
                  max_depth=5,
                  min_samples_leaf=2,
                  max_features='sqrt',
                  random_state=0))
], memory=cachedir)


In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

# R² on test
y_pred = pipeline.predict(X_test)
print("Test  R²:", r2_score(y_test, y_pred))

# Cross-validated R² on train
cv_scores = cross_val_score(pipeline, X_train, y_train,
                            cv=5, scoring='r2')
print(f"CV R²: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")


# New Data Prediction

In [ ]:
new_data = X_test.iloc[[0]]  

# predict
prediction = pipeline.predict(new_data)

print(prediction)

## Pickling the Model file for Deployment

In [ ]:
import pickle

In [ ]:
# Save the entire pipeline, not just the regression model
with open('regmodel.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
pickled_model = pickle.load(open('regmodel.pkl', 'rb'))

In [ ]:
pickled_model.predict(new_data)